# Washington DC Biking data | Hourly Bike Count Prediction

# 3. Modeling for WorkingDays
MBD O-1-5

In [18]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:56140 Dashboard: http://127.0.0.1:56141/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [8]:
# Load datasets
wd_h = dd.read_csv('./workingdays_data_prepared.csv')
wd_h['dteday']=dd.to_datetime(wd_h.dteday ,unit='ns')

In [9]:
wd_h.compute().head()

,instant,dteday,yr,holiday,workingday,temp,atemp,hum,windspeed,casual,...,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,hr_2,hr_3
0,48,2011-01-03,0,0,1,0.22,0.1970,0.44,0.3582,0,...,0,0,0,0,0,0,0,0,0,0
1,49,2011-01-03,0,0,1,0.20,0.1667,0.44,0.4179,0,...,0,0,0,0,0,0,0,0,0,0
2,50,2011-01-03,0,0,1,0.16,0.1364,0.47,0.3881,0,...,0,0,0,0,0,0,0,0,0,0
3,51,2011-01-03,0,0,1,0.16,0.1364,0.47,0.2836,0,...,0,0,0,0,0,0,0,0,0,0
4,52,2011-01-03,0,0,1,0.14,0.1061,0.50,0.3881,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
wd_h.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11865 entries, 0 to 11864
Data columns (total 73 columns):
instant                  11865 non-null int64
dteday                   11865 non-null datetime64[ns]
yr                       11865 non-null int64
holiday                  11865 non-null int64
workingday               11865 non-null int64
temp                     11865 non-null float64
atemp                    11865 non-null float64
hum                      11865 non-null float64
windspeed                11865 non-null float64
casual                   11865 non-null int64
registered               11865 non-null int64
cnt                      11865 non-null int64
isDaylight               11865 non-null bool
isNoon                   11865 non-null bool
relative_temp            11865 non-null float64
relative_atemp           11865 non-null float64
relative_hum             11865 non-null float64
relative_windspeed       11865 non-null float64
Wday_RushHour_High       11865 non-null 

## Subset Data

Use "train_test_split" for dask to subset the train vs test data failed, therefore the . dataset is split manually using the "dteday" from Q1/2011 to Q2/2012 - Train & Q3-Q4/2012 - test.

In [11]:
#X_train, X_test, y_train, y_test = train_test_split(X, y) #Note that does not workl for train_test_split from dask-ml

In [12]:
X_train = wd_h[wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday','instant'],axis=1)
X_train.compute().head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,isDaylight,isNoon,relative_temp,...,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,hr_2,hr_3
0,0,0,1,0.22,0.1970,0.44,0.3582,False,False,-1.611902,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0.20,0.1667,0.44,0.4179,False,False,-1.790420,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0.16,0.1364,0.47,0.3881,False,False,-2.180605,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0.16,0.1364,0.47,0.2836,False,False,-2.061394,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0.14,0.1061,0.50,0.3881,False,False,-2.177167,...,0,0,0,0,0,0,0,0,0,0


In [13]:
Y_train =wd_h['cnt'][wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)]
Y_train.compute().head()

0     5
1     2
2     1
3     3
4    30
Name: cnt, dtype: int64

In [14]:
x_test = wd_h[wd_h['dteday'] > datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday','instant'],axis=1)
x_test.compute().head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,isDaylight,isNoon,relative_temp,...,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,hr_2,hr_3
8976,1,0,1,0.78,0.7424,0.62,0.1343,False,False,-0.209613,...,0,0,0,0,0,0,0,0,0,0
8977,1,0,1,0.78,0.7273,0.55,0.2239,False,False,-0.212456,...,0,0,0,0,0,0,0,0,0,0
8978,1,0,1,0.76,0.7121,0.62,0.1343,False,False,-0.434529,...,0,0,0,0,0,0,0,0,1,0
8979,1,0,1,0.74,0.6970,0.70,0.1940,False,False,-0.657493,...,0,0,0,0,0,0,0,0,0,1
8980,1,0,1,0.74,0.6970,0.66,0.2239,False,False,-0.659466,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y_test =wd_h['cnt'][wd_h['dteday'] > datetime.datetime(2012, 7, 5, 0, 0)]
y_test.compute().head()

8976    102
8977     29
8978     17
8979      7
8980      6
Name: cnt, dtype: int64

In [16]:
X_train = X_train#.values
Y_train = Y_train#.values

x_test = x_test#.values
y_test = y_test#.values

## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

Linear Regression using dask provides error "TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'" with the same code that ran for baseline.

In [17]:
# from sklearn.metrics import r2_score

# #X_Train, Y_train = make_regression()
# lrd = LinearRegression() #Linear regression with dask_ml
# lrd.fit(X_train.values, Y_train.values)
# y_pred = lrd.predict(X_Test.values)
# r2_score(y_test.compute(), y_pred.compute())

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

##Linear Regression
lm_parameters = {'fit_intercept':[True,False] ,'normalize':[True,False]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv ,return_train_score=True)
with joblib.parallel_backend('dask'):
    lm.fit(X_train, Y_train)
lm.cv_results_
lm_predictions_w = lm.predict(x_test)
lm.score(x_test, y_test)

0.7742681424381046

Starting with the simplest model on Q3 , even though the score is high here this is the prediction for Q3 (FALL) , as we saw in the EDA FALL SPRING AND SUMMER behaved in the same way but winter was a bit different so we can say that the model learned from 5 similar season and we would not expect the same score for Q4 ( winter ) . This clearly an indicator that the feature created previously have helped our model to learn more about the target

In [35]:
# X_Train, y_train = make_regression()
lr = LinearRegression()
lr.fit(X_Train, y_train)
lr.predict(X_Test)
lr.score(X_Test.compute(), y_test.compute())

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

### Linear Regression using SKLEARN & Dask Dataframes

In [18]:
X_Train = X_Train_2011_2012Q2
y_train = Y_cnt_train_2011_2012Q2

X_Test = X_Test_2012Q3
y_test = Y_cnt_test_2012Q3

In [17]:
##Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib


lm_parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True], "n_jobs": [-1]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv ,return_train_score=True)

with joblib.parallel_backend('dask'):
    lm.fit(X_Train, y_train)
    lm.cv_results_
    lm_predictions = lm.predict(X_Test).compute()
    lm.score(X_Test.compute(), y_test.compute())



TypeError: 'float' object cannot be interpreted as an integer

In [1]:
# digits = load_digits()

# param_space = {
#     'C': np.logspace(-6, 6, 13),
#     'gamma': np.logspace(-8, 8, 17),
#     'tol': np.logspace(-4, -1, 4),
#     'class_weight': [None, 'balanced'],
# }

# model = SVC(kernel='rbf')
# search = RandomizedSearchCV(model, param_space, cv=3, n_iter=50, verbose=10)

# with joblib.parallel_backend('dask'):
#     search.fit(digits.data, digits.target)

NameError: name 'load_digits' is not defined

## Random Forest w/ GS for *Weekdays* for 2011 -> 2012Q3

After trying linear regressor we tried random forest with a large grid search in order to make sure that we are using the best possible parameters for the model 
Our expectations were to have a higher score then the linear model but we did not, This could be due to the fact that there are alot of linearly correlated target to the data such as the new weather variable created and the 
mean_per_hour which reflects a direct linear correlation of the past target in the same hour.
Random forest acts best when the data is clearly partitione in groups such as the Rush_Hour and Is_Daylight that clearly differentiate the different target values

In [6]:
from sklearn.ensemble import RandomForestRegressor


RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                             "n_jobs": [-1],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
rf.cv_results_
rf_predictions = lm.predict(X_Test_2012Q3)
rf.score(X_Test_2012Q3, Y_cnt_test_2012Q3)


0.9289808963621973

## XGBoost regressor w/ GS for *Holidays* for 2011 -> 2012Q3

XGBoost is known for winning most of the kaggle competitions so we thought it was worth a shot , specialy since we know that this algorithm work on the errors of a tree algorithm and improves on it
 the Grid Search used wasa wide enough in terms of parameter and gave back the best scores between all the algortihms

In [9]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300], 'random_state':[random_seed]
}

model = xgb.XGBRegressor()

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)
xg.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
xg_predictions = xg.predict(X_Test_2012Q3)
xg.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 29.8min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 37.2min finished


0.934419033223012

## Subset Q4 data and concat it with its level-0 predctions

Taking the original Q4 data and adding to it the predictions from the 3 algortihms chosen 

In [11]:
#Get Q4 data
X_Test_Q4 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_Q4 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))]
oringal_cnt = Y_cnt_test_Q4

#get level-0 predictions for Q4 data
Q4_lm_predictions = pd.DataFrame(lm.predict(X_Test_Q4), columns=["lm_predictions"])
Q4_rf_predictions = pd.DataFrame(rf.predict(X_Test_Q4), columns=["rf_predictions"])
Q4_xg_predictions = pd.DataFrame(xg.predict(X_Test_Q4), columns=["xg_predictions"])

X_Test_Q4.reset_index(drop=True,inplace=True)
Q4_lm_predictions.reset_index(drop=True,inplace=True)
Q4_rf_predictions.reset_index(drop=True,inplace=True)
Q4_xg_predictions.reset_index(drop=True,inplace=True)
Y_cnt_test_Q4.reset_index(drop=True,inplace=True)

#concat Q4 with level-0 predictions 
X_Test_Q4_with_predictions = pd.concat([Q4_lm_predictions, Q4_rf_predictions,Q4_xg_predictions, X_Test_Q4 ], axis=1)
Y_cnt_test_Q4 = pd.DataFrame({"target": Y_cnt_test_Q4})

## Q4 scores

###### Here are the scores of the trained algorithms on the Q4 data in order to be able to compare our initial results to the stacking ones later on

Linear Regression :

In [12]:
lm.score(X_Test_Q4, Y_cnt_test_Q4)

0.8839966859540709

Random Forest :

In [13]:
rf.score(X_Test_Q4, Y_cnt_test_Q4)

0.8715926918607073

XGBoost :

In [14]:
xg.score(X_Test_Q4, Y_cnt_test_Q4)

0.8980765759281988

## Train and test Q4

### XGBoost regressor w/ GS for Level 1 *Holidays* for 2011 -> 2012Q3

According to the scores above  XGBoost will be retrained on the 3rd Quarter with the predictions of the best 3 models and will be tested on Q4  with the predictions on Q4 

In [19]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
           "n_jobs": [-1],
          'n_estimators': [100, 300], 'random_state':[random_seed]
}

model = xgb.XGBRegressor()

xg1 = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'explained_variance',
                    verbose=True)

xg1.fit(combinedPredictions, combinedPredictionsTarget)
xg1_predictions = xg1.predict(X_Test_Q4_with_predictions)
xg1.score(X_Test_Q4_with_predictions, Y_cnt_test_Q4)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 204 tasks      | elapsed:   36.6s
[Parallel(n_jobs=-1)]: Done 454 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1254 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  5.9min finished


0.9010145680993351

This is the final score for the prediction of Q4 workingdays : 0.9022288970285075

Things that could have increased the score more :
    - CHoosing the right amount of features for every algorithms , but this was very computationaly heavy , since trying an RFECV with the already time ocnsuming algorithms would take too long
    - Increasing the gridsearch or maybe using Bayesian Optimization 
Things we thought would increase our score that we tried but did not work :
    - Running the level 1 algorithm only on the predictions without the initial data 
    - Doing stacking in a way that would use the prediction of the TimeSeriesCross Validation and then retrain the level 0 algorithm with more data 
    (it did not work because the prediction in each fold used was being outputed from a different "Train" of the same algorithm , and the prediction for test data would only be predicted by the training of one of the folds, 
    this means that what the level1 algorithm is trying to learn from the predicted values would be different in each fold )
    

### Save Predictions for Weekdays

In [43]:
work_predictions  = pd.concat( (pd.DataFrame(xg1.predict(X_Test_Q4_with_predictions)), Y_cnt_test_Q4), axis =1 )
dates = wd_h['dteday'][(wd_h['dteday'] >= datetime.datetime(2012, 9, 30, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))]
dates=pd.to_datetime(dates, format='%Y-%m-%d')
dates.reset_index(drop=True,inplace=True)
work_predictions.reset_index(drop=True,inplace=True)
full_wd=pd.concat([dates,work_predictions],axis=1)
full_wd.to_csv("pred_working.csv", index=False)